# SPR 2026 - Logistic Regression v3 (RandomizedSearch)

**Score baseline (v2):** 0.72935

**Melhorias:**
- RandomizedSearchCV para tuning
- Class weights balanceados
- Threshold tuning por classe
- SMOTE para classes 5/6 (opcional)
- Flag para remover classe 2

---
**CONFIGURACAO KAGGLE:** Internet OFF
---

In [ ]:
import numpy as npimport pandas as pdfrom sklearn.feature_extraction.text import TfidfVectorizerfrom sklearn.linear_model import LogisticRegressionfrom sklearn.model_selection import RandomizedSearchCV, StratifiedKFoldfrom scipy.stats import loguniformimport warningswarnings.filterwarnings('ignore')print("="*60)print("SPR 2026 - Logistic Regression v3 (RandomizedSearch)")print("="*60)SEED = 42DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'np.random.seed(SEED)# FLAGS - AJUSTE AQUIREMOVE_CLASS_2 = FalseUSE_SMOTE = FalseUSE_THRESHOLD_TUNING = TrueN_SEARCH_ITER = 30# Dadosprint("\n[1/5] Carregando dados...")train = pd.read_csv(f'{DATA_DIR}/train.csv')test = pd.read_csv(f'{DATA_DIR}/test.csv')print(f"Train: {train.shape} | Test: {test.shape}")if REMOVE_CLASS_2:    train = train[train['target'] != 2].reset_index(drop=True)    print(f"Sem classe 2: {train.shape}")# TF-IDFprint("\n[2/5] TF-IDF...")tfidf = TfidfVectorizer(max_features=15000, ngram_range=(1,2), min_df=2, max_df=0.95, sublinear_tf=True)X_train = tfidf.fit_transform(train['report'])X_test = tfidf.transform(test['report'])y_train = train['target'].valuesprint(f"Shape: {X_train.shape}")# SMOTE opcionalif USE_SMOTE:    try:        from imblearn.over_sampling import SMOTE        smote = SMOTE(sampling_strategy={5:500, 6:500}, random_state=SEED, k_neighbors=3)        X_train, y_train = smote.fit_resample(X_train, y_train)        print(f"SMOTE: {X_train.shape}")    except ImportError:        print("imblearn nao disponivel")# RandomizedSearchprint(f"\n[3/5] RandomizedSearchCV ({N_SEARCH_ITER} iter)...")param_dist = {    'C': loguniform(0.01, 100),    'penalty': ['l1', 'l2'],    'solver': ['saga'],    'max_iter': [500, 1000, 2000],}base = LogisticRegression(class_weight='balanced', random_state=SEED, n_jobs=-1)cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)search = RandomizedSearchCV(base, param_dist, n_iter=N_SEARCH_ITER, cv=cv, scoring='f1_macro', n_jobs=-1, random_state=SEED, verbose=1)search.fit(X_train, y_train)print(f"Best params: {search.best_params_}")print(f"Best F1-macro: {search.best_score_:.4f}")best_model = search.best_estimator_# Threshold tuningif USE_THRESHOLD_TUNING:    print("\n[4/5] Threshold tuning...")    proba = best_model.predict_proba(X_test)    classes = best_model.classes_    thresholds = {0:0.5, 1:0.5, 2:0.5, 3:0.5, 4:0.5, 5:0.35, 6:0.35}    preds = []    for i in range(len(proba)):        adj = proba[i].copy()        for j,c in enumerate(classes):            if c in thresholds:                adj[j] *= (0.5/thresholds[c])        preds.append(classes[np.argmax(adj)])    predictions = np.array(preds)else:    print("\n[4/5] Predicao padrao...")    predictions = best_model.predict(X_test)# Submissaoprint("\n[5/5] Gerando submissao...")submission = pd.DataFrame({'ID': test['ID'], 'target': predictions})submission.to_csv('submission.csv', index=False)print("="*60)print("CONCLUIDO!")print("="*60)print("\nDistribuicao:")print(submission['target'].value_counts().sort_index())